# 01_baseline Playground

이 노트북은 **Pure Naive RAG (V0)** 파이프라인을 실험하는 공간입니다.
메타데이터 필터링 없이, 단순한 Vector Search 기반의 검색 및 답변 생성을 테스트합니다.

**사전 준비**:
1. `.env` 파일에 HyperCLOVA X 및 Milvus 설정 완료
2. `python 01_baseline/ingest.py` 실행하여 데이터 적재 완료

In [1]:
# Auto-reload ensures that if you edit pipeline.py, changes are reflected here immediately
%load_ext autoreload
%autoreload 2

import sys
import os
from dotenv import load_dotenv

load_dotenv()

# Add root directory to sys.path to import config
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import the pipeline class from our structured file
from pipeline import RAGPipelineV0
from config import Config

## 1. Milvus 적재 상태 확인
데이터가 잘 들어갔는지 확인해봅니다.

In [2]:
from pymilvus import MilvusClient

client = MilvusClient(uri=Config.MILVUS_URI, token=Config.MILVUS_TOKEN)
collection_name = Config.MILVUS_COLLECTION_NAME_V0

if client.has_collection(collection_name):
    count = client.query(collection_name=collection_name, filter="", output_fields=["count(*)"])[0]["count(*)"]
    print(f"Collection '{collection_name}' exists with {count} entities.")
else:
    print(f"Collection '{collection_name}' does not exist. Please run ingest.py first.")

Collection 'data_v1' exists with 20458 entities.


## 2. 파이프라인 초기화 및 테스트

In [3]:
pipeline = RAGPipelineV0()
print("Pipeline Initialized!")

Pipeline Initialized!


### 2.1. 단순 검색 테스트 (Retrieval Only)
질문에 대해 어떤 문서들이 검색되는지 확인합니다.

In [4]:
query = "한국철도공사의 예산 관련 지적사항은?"
retrieved_docs = pipeline.retriever.invoke(query)

for i, doc in enumerate(retrieved_docs):
    print(f"[{i+1}] {doc.metadata.get('title', 'No Title')} (Score: N/A in LangChain retriever)")
    print(f"Preview: {doc.page_content[:100]}...\n")

[1] No Title (Score: N/A in LangChain retriever)
Preview: [Action]
조치할 사항 한국철도공사 사장 은 앞으로 임금체계 개편 등을 하여 ｢ 공기업 ･ 준정부기관 예 산편성지침 ｣ 등과 다르게 경영평가성과급 및 내부평가급의 지급기준을 ...

[2] No Title (Score: N/A in LangChain retriever)
Preview: [Content]
감 사 원 통 보 제 목 징계 관련 내부규정 정비 미흡 소 관 기 관 국가철도공단 조 치 기 관 국가철도공단 내 용 1. 업무 개요 국가 철도공단 ( 이하 “ 공...

[3] No Title (Score: N/A in LangChain retriever)
Preview: [Content]
통보, 경고 요구 제 목 ◍◍시설물사용료과다납부 소 관 부 서□□본부○○처 조 치 부 서□□본부○○처 내 용 1.업무개요 □□본부○○처(이하“위부서”라한다.)는◍...

[4] No Title (Score: N/A in LangChain retriever)
Preview: [Content]
시정 요구 및 권고⋅통보 제 목철도CY 위탁운영 및 관리에 관한 사항 소 관 부 서◍◍◍◍ 본부, ◓◓본부 조 치 부 서 ○○○○ 본부, ◓◓본부 내 용 1. 철...

[5] No Title (Score: N/A in LangChain retriever)
Preview: [Content]
개선요구 ㆍ통보 제 목 철도건설사업종합배치도작성기준관리소홀 소 관 부 서 **본부 조 치 부 서 ①**본부(**처)②**본부(**처) 내 용 1.업무개요 **본부...



### 2.2. 전체 답변 생성 테스트 (Generation)
실제 RAG 답변을 생성해봅니다.

In [5]:
answer = pipeline.run(query)
print("\n=== Answer ===\n")
print(answer)

Processing query (V0): 한국철도공사의 예산 관련 지적사항은?

=== Answer ===

한국철도공사와 관련하여 지적된 예산 관련 사항은 다음과 같습니다:

1. **경영평가성과급 및 내부평가급 과다 지급 방지**: 한국철도공사 사장은 '공기업·준정부기관 예산편성지침'과 다르게 경영평가성과급 및 내부평가급의 지급 기준을 높여 경영평가성과급 등을 과다하게 지급하는 일이 없도록 관련 업무를 철저히 해야 합니다.

2. **과다 납부된 시설물 사용료 회수 및 관련자 경고**: □□본부장은 공단에 과다 납부된 ◍◍시설물 사용료 ***천원을 회수하도록 시정 조치하고, 업무를 소홀히 한 관련자에게 경고를 요구해야 합니다.
